First we bring in all of our imports and set the desired size of the image. We have chosen 28x28 as the default.

In [15]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras import layers
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot as plt
import itertools

import warnings
warnings.filterwarnings('ignore')

# SET THE IMAGE SIZE
IMAGE_SIZE = 28

Next we define a method to bring in all of our images from either the processed or unprocessed folders.

In [16]:
def get_data(train_test, processed_unprocessed):
  cwd = os.getcwd()
  data = []

  path = os.path.realpath(f'processed_images/{train_test}/{processed_unprocessed}/')
  for filename in os.listdir(path):
      #Read in Image
      filepath = f"{path}/{filename}"

      img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

      #Resize Image
      desired_size = IMAGE_SIZE
      old_size = img.shape[:2] 

      ratio = float(desired_size)/max(old_size)
      new_size = tuple([int(x*ratio) for x in old_size])

      img = cv2.resize(img, (new_size[1], new_size[0]))

      delta_w = desired_size - new_size[1]
      delta_h = desired_size - new_size[0]
      top, bottom = delta_h//2, delta_h-(delta_h//2)
      left, right = delta_w//2, delta_w-(delta_w//2)

      color = [255, 255, 255]
      img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

      data.append([img, filename[0]])

  return data

Next, we ask the user which dataset they would like to load in. The set processed into a binarized image, or the grayscale unprocessed version of the dataset.

In [17]:
choices = ["binary", "grayscale"]
user_input = ""

print("Which dataset would you like to train on?")
while user_input not in choices:
  user_input = input("\'binary\'/\'grayscale\'?:")

train_data = get_data("train", user_input)
test_data = get_data("test", user_input)

Which dataset would you like to train on?
'binary'/'grayscale'?:grayscale


Once we have our data loaded into memory we proceed to split the data into its training and test sets and append all of the class labels to the corresponding images. Since our dataset is small we have chosen to use a validation set that is only 5% of our training data. We also stratify the split to ensure an equal distribution of class labels in our validation set.

In [18]:
X_train = []
Y_train = []
X_test = []
Y_test = []

for feature, label in train_data:
  X_train.append(feature)
  Y_train.append(label)

for feature, label in test_data:
  X_test.append(feature)
  Y_test.append(label)

# Normalize the data
X_train = np.array(X_train) / 255.0
X_test = np.array(X_test) / 255.0

X_train = X_train.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
X_test = X_test.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
Y_train = to_categorical(Y_train, num_classes = 10)
Y_test = to_categorical(Y_test, num_classes= 10)

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, stratify=Y_train, test_size = 0.05)

Next we build out our CNN model, compile it and print out a summary of its layers for visualization purposes.

In [19]:
model = Sequential()

model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (IMAGE_SIZE,IMAGE_SIZE,1)))
model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation = "softmax"))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer=optimizer , loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)         

Here we perform our final setps before handing off all the data to our model for training. We choose to set the epochs to 64 and the batch size to 12, but these can be adjusted and played with. Since our dataset was relatively small we decided to bolster the dataset using image augmentations. We also added a learning rate annealer to help prevent overfitting and end training early if it isn't seeing an imrovement in our loss per epoch.

In [20]:
epochs = 64 
batch_size = 12

learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

datagen = ImageDataGenerator(
        rotation_range=12,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1)  # randomly shift images vertically (fraction of total height)

datagen.fit(X_train)

Finally, we fit our model using our ImageDataGenerator, learning rate annealer, and validation set.

In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              validation_data = (X_val, Y_val),
                              epochs = epochs, 
                              steps_per_epoch=X_train.shape[0] // batch_size, 
                              callbacks=[learning_rate_reduction])

Epoch 1/64
83/83 [==============================] - 1s 10ms/step - loss: 2.3146 - accuracy: 0.1057 - val_loss: 2.3026 - val_accuracy: 0.0943
Epoch 2/64
83/83 [==============================] - 1s 9ms/step - loss: 2.3050 - accuracy: 0.1051 - val_loss: 2.2959 - val_accuracy: 0.1321
Epoch 3/64
83/83 [==============================] - 1s 9ms/step - loss: 2.2006 - accuracy: 0.1849 - val_loss: 1.3259 - val_accuracy: 0.5849
Epoch 4/64
83/83 [==============================] - 1s 9ms/step - loss: 1.6464 - accuracy: 0.4638 - val_loss: 1.3092 - val_accuracy: 0.5472
Epoch 5/64
83/83 [==============================] - 1s 9ms/step - loss: 1.4753 - accuracy: 0.5087 - val_loss: 0.7924 - val_accuracy: 0.7358
Epoch 6/64
83/83 [==============================] - 1s 9ms/step - loss: 1.1617 - accuracy: 0.6078 - val_loss: 0.7332 - val_accuracy: 0.8302
Epoch 7/64
83/83 [==============================] - 1s 9ms/step - loss: 1.0738 - accuracy: 0.6209 - val_loss: 0.5848 - val_accuracy: 0.8491
Epoch 8/64
83/83 [=

We can then plot out some metrics recorded from our traing to ensure that our ROC looks appropriate.

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

print(history.history.keys())

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

Next, we can test our trained model on the test set and create a classification matrix so that we can visualize our results a little better.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(X_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

We can also print out a classification report so that we can see how our model performed on making classifications for each class based on a set of statistics.

In [ ]:
print(classification_report(Y_true, Y_pred_classes))

Finally, we can print out our top 4 misclassified labels to better visualize why they were mistaken.

In [ ]:
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_test_errors = X_test[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    n = 0
    nrows = 2
    ncols = 2
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    fig.set_figheight(10)
    fig.set_figwidth(10)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((IMAGE_SIZE,IMAGE_SIZE)))
            ax[row,col].set_title(f" Predicted label :{pred_errors[error]}\nTrue label :{obs_errors[error]}")
            n += 1

# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 4 errors 
most_important_errors = sorted_dela_errors[-4:]

# Show the top 4 errors
display_errors(most_important_errors, X_test_errors, Y_pred_classes_errors, Y_true_errors)